In [108]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [110]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
2
0
NVIDIA GeForce RTX 3090


In [111]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [112]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/mountHDD1/LanxHuyen/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
block_splits_all = '/media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth' 
eeg_raw = '/media/mountHDD1/LanxHuyen/CVPR2017/eeg_55_95_std.pth'
# block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
# print(os.listdir(torch_models_dir))

In [113]:
# eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
# splits_all_path = os.path.join(torch_models_dir, block_splits_all)
eeg_dataset = eeg_raw
splits_all_path = block_splits_all
# splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
# print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)
print(eeg_dataset,'\n', splits_all_path)

/media/mountHDD1/LanxHuyen/CVPR2017/eeg_55_95_std.pth 
 /media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth


In [114]:
splits_all = torch.load(splits_all_path)
# splits_single = torch.load(splits_single_path)

In [115]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [116]:
# # print(splits_single)
# # print(splits_single['splits'])
# print(len(splits_single['splits'][0]['train']))
# print(len(splits_single['splits'][0]['val']))
# print(len(splits_single['splits'][0]['test']))

In [117]:
eeg_loaded = torch.load(eeg_dataset)

In [119]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))
print(len(means))
print(len(stddevs))
# print(means)
# print(stddevs)
print(*labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[2])



3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
128
128
n02389026 n03888257 n03584829 n02607072 n03297495 n03063599 n03792782 n04086273 n02510455 n11939491 n02951358 n02281787 n02106662 n04120489 n03590841 n02992529 n03445777 n03180011 n02906734 n07873807 n03773504 n02492035 n03982430 n03709823 n03100240 n03376595 n03877472 n03775071 n03272010 n04069434 n03452741 n03792972 n07753592 n13054560 n03197337 n02504458 n02690373 n03272562 n04044716 n02124075
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-1.8615e-02, -2.2027e-01, -3.8487e-01,  ...,  5.3082e-01,
          3.0621e-01,  2.6092e-02],
        [-3.1586e-02, -2.2723e-01, -3.8992e-01,  ...,  5.4794e-01,
          3.0803e-01,  1.7300e-02],
        [ 3.8073e-02,  4.0159e-01,  7.0329e-01,  ..., -7.8495e-01,
         -4.3544e-01, -2.3261e-02],
        ...,
        [-2.1302e-02,  1.1708e-01,  2.2351e-01,  ...,  3.8769e-01,
          2.0366e-01, -2.0062e-02],
        [-8.4068e-04, -5.9282e-03, -1.0319e-02,  ...,  3.1338

In [120]:
import numpy as np
eeg_length = np.zeros(len(dataset))
for i in range(len(dataset)):
    eeg_length[i] = list(dataset[i]['eeg'].size())[1]

In [121]:
np.min(eeg_length)
print(eeg_length[1000:1200])

[501. 499. 531. 499. 509. 500. 499. 529. 516. 514. 498. 500. 494. 508.
 516. 505. 498. 497. 531. 500. 497. 500. 530. 528. 500. 498. 506. 513.
 499. 536. 502. 521. 528. 520. 528. 513. 515. 598. 501. 497. 499. 499.
 494. 502. 530. 535. 546. 529. 497. 514. 529. 528. 516. 516. 514. 499.
 500. 514. 517. 498. 496. 494. 517. 500. 513. 502. 498. 514. 499. 497.
 504. 515. 499. 509. 496. 498. 502. 498. 534. 503. 531. 496. 496. 516.
 500. 498. 530. 514. 499. 529. 492. 544. 499. 514. 512. 504. 499. 517.
 495. 498. 546. 498. 518. 540. 498. 500. 500. 504. 543. 501. 499. 524.
 515. 504. 495. 496. 544. 500. 499. 496. 498. 516. 515. 526. 513. 499.
 502. 516. 544. 501. 648. 529. 497. 517. 515. 519. 516. 531. 494. 530.
 532. 529. 496. 499. 496. 502. 519. 514. 499. 499. 531. 545. 498. 498.
 517. 493. 499. 499. 502. 504. 518. 500. 529. 499. 496. 498. 501. 533.
 511. 517. 514. 512. 498. 530. 507. 499. 546. 514. 514. 531. 501. 501.
 501. 500. 499. 496. 500. 502. 527. 515. 497. 497. 528. 530. 515. 512.
 518. 

In [122]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/mountHDD1/LanxHuyen/CVPR2017/eeg_55_95_std.pth', model_type='model10', splits_path='/media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier=None)


In [123]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name=split) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [125]:
# Options
opt.classifier = "LSTM4"
opt.batch_size = 64
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [126]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [127]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
# for i, (input, target) in enumerate(loaders["train"]):
#     if i<20:
#         print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
for i in range(0, 40):
    eeg, label_val = splitter["val"][i]
    eeg, label_train = splitter["train"][i]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [125, 32, 32]
1: Label val: 37; label train: 10
2: Label val: 38; label train: 10
3: Label val: 11; label train: 30
4: Label val: 10; label train: 25
5: Label val: 7; label train: 18
6: Label val: 35; label train: 3
7: Label val: 13; label train: 8
8: Label val: 2; label train: 11
9: Label val: 14; label train: 18
10: Label val: 2; label train: 28
11: Label val: 33; label train: 38
12: Label val: 26; label train: 20
13: Label val: 34; label train: 3
14: Label val: 18; label train: 28
15: Label val: 11; label train: 23
16: Label val: 32; label train: 0
17: Label val: 6; label train: 34
18: Label val: 22; label train: 20
19: Label val: 24; label train: 23
20: Label val: 24; label train: 39
21: Label val: 29; label train: 0
22: Label val: 3; label train: 34
23: Label val: 25; label train: 21
24: Label val: 11; label train: 39
25: Label val: 22; label train: 6
26: Label val: 36; label train: 26
27: Label val: 30; label train: 20
28

In [128]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_LSTM(
  (lstm): LSTM(128, 128, batch_first=True)
  (output1): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM                          [1, 40]                   --
├─LSTM: 1-1                              [1, 440, 128]             132,096
├─Linear: 1-2                            [1, 128]                  16,512
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 153,768
Trainable params: 153,768
Non-trainable params: 0
Total mult-adds (M): 58.14
Input size (MB): 0.23
Forward/backward pass size (MB): 0.45
Params size (MB): 0.62
Estimated Total Size (MB): 1.29

In [130]:
model_path = (   f"CVPR2017-{opt.classifier}-{length}-{channel}-55_95")
print(model_path)

CVPR2017-LSTM4-440-128-55_95


In [131]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/mountHDD1/LanxHuyen/CVPR2017/eeg_55_95_std.pth', model_type='model10', splits_path='/media/mountHDD1/LanxHuyen/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=64, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='LSTM4')


In [132]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    results = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path,
            print_every_train = 100,
            print_every_val = 125)

Epoch 1
Train Batch 100 (every 100 batch): Loss=3.6049; accuracy=0.0427
Epoch 1 summary: train_loss: 3.6567 | train_acc: 0.0448 | val_loss: 3.5636 | val_acc: 0.0547
Epoch 2
Train Batch 100 (every 100 batch): Loss=3.2222; accuracy=0.0733
Epoch 2 summary: train_loss: 3.3790 | train_acc: 0.0742 | val_loss: 3.3075 | val_acc: 0.0713
Epoch 3
Train Batch 100 (every 100 batch): Loss=3.0810; accuracy=0.0978
Epoch 3 summary: train_loss: 3.2074 | train_acc: 0.0985 | val_loss: 3.1985 | val_acc: 0.0954
Epoch 4
Train Batch 100 (every 100 batch): Loss=2.7935; accuracy=0.1267
Epoch 4 summary: train_loss: 2.9133 | train_acc: 0.1282 | val_loss: 2.9966 | val_acc: 0.0962
Epoch 5
Train Batch 100 (every 100 batch): Loss=2.9182; accuracy=0.1187
Epoch 5 summary: train_loss: 3.0469 | train_acc: 0.1215 | val_loss: 3.0266 | val_acc: 0.1018
Epoch 6
Train Batch 100 (every 100 batch): Loss=2.7205; accuracy=0.1514
Epoch 6 summary: train_loss: 2.7690 | train_acc: 0.1499 | val_loss: 2.8514 | val_acc: 0.1184
Epoch 7
Tr

Train Batch 100 (every 100 batch): Loss=0.4704; accuracy=0.8187
Train Batch 200 (every 100 batch): Loss=0.2372; accuracy=0.8181
Train Batch 300 (every 100 batch): Loss=0.2904; accuracy=0.8152
Train Batch 400 (every 100 batch): Loss=0.2112; accuracy=0.8194
Epoch 70 summary: train_loss: 0.5636 | train_acc: 0.8289 | val_loss: 5.8814 | val_acc: 0.2016
Epoch 71
Train Batch 100 (every 100 batch): Loss=0.3092; accuracy=0.9256
Train Batch 200 (every 100 batch): Loss=0.3137; accuracy=0.9300
Train Batch 300 (every 100 batch): Loss=0.0731; accuracy=0.9383
Train Batch 400 (every 100 batch): Loss=0.2065; accuracy=0.9442
Epoch 71 summary: train_loss: 0.1714 | train_acc: 0.9450 | val_loss: 5.7815 | val_acc: 0.2046
Epoch 72
Train Batch 100 (every 100 batch): Loss=0.0299; accuracy=0.9656
Train Batch 200 (every 100 batch): Loss=0.0485; accuracy=0.9716
Train Batch 300 (every 100 batch): Loss=0.0297; accuracy=0.9727
Train Batch 400 (every 100 batch): Loss=0.0633; accuracy=0.9722
Epoch 72 summary: train_lo

Train Batch 100 (every 100 batch): Loss=0.0293; accuracy=0.9712
Train Batch 200 (every 100 batch): Loss=0.0162; accuracy=0.9678
Train Batch 300 (every 100 batch): Loss=0.1756; accuracy=0.9677
Train Batch 400 (every 100 batch): Loss=0.0787; accuracy=0.9627
Epoch 93 summary: train_loss: 0.1430 | train_acc: 0.9585 | val_loss: 6.7716 | val_acc: 0.2149
Epoch 94
Train Batch 100 (every 100 batch): Loss=0.0577; accuracy=0.9700
Train Batch 200 (every 100 batch): Loss=0.1885; accuracy=0.9656
Train Batch 300 (every 100 batch): Loss=0.1342; accuracy=0.9652
Train Batch 400 (every 100 batch): Loss=0.3027; accuracy=0.9586
Epoch 94 summary: train_loss: 0.1505 | train_acc: 0.9534 | val_loss: 7.1981 | val_acc: 0.1959
Epoch 95
Train Batch 100 (every 100 batch): Loss=0.1803; accuracy=0.9500
Train Batch 200 (every 100 batch): Loss=0.0802; accuracy=0.9528
Train Batch 300 (every 100 batch): Loss=0.0852; accuracy=0.9508
Train Batch 400 (every 100 batch): Loss=0.0628; accuracy=0.9506
Epoch 95 summary: train_lo

In [ ]:
# val = results["val_acc"]
# # test = results["test_acc"]

# print("Validation accuracy: ", val)
# # print("Test accuracy: ", test)

In [ ]:
print(accuracy_val)